# Task
Langchain, Langgraph, Tavily Search를 사용하여 다음 기능을 포함하는 딥 리서치 도구를 개발합니다.
1. 검색이 필요하지 않으면 검색을 수행하지 않습니다.
2. 검색 결과는 최소 2개에서 최대 10개까지 활용합니다.
3. 보고서 작성 목표를 생성하는 노드를 포함합니다.
4. 목표 달성 여부를 확인하는 노드를 포함합니다.

# 실습 코드


### 환경 셋팅

In [1]:
!pip install -qU langchain_openai langchain-community pypdf langgraph tavily-python langchain-tavily langfuse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.8 MB/s eta 0:00:00


In [2]:
from __future__ import annotations

import os
import time
from typing import List, Dict, Any
from typing_extensions import TypedDict

from pydantic import BaseModel, Field

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END



### key 설정

In [3]:
# .env 파일에서 로드 (시크릿은 코드에 넣지 말고 .env에 작성)
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### LLM 및 검색 도구 초기화

In [4]:
llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.2,
    max_retries=2,
    api_key=OPENAI_API_KEY,
)

from langchain_tavily import TavilySearch
tavily_tool = TavilySearch(
    max_results=5,
    topic="general",
    search_depth="advanced",
    include_raw_content=True,
    tavily_api_key=TAVILY_API_KEY,
)

### 상태 정의 및 유틸 함수

In [5]:
class GraphState(TypedDict, total=False):
    question: str
    need_search: bool
    queries: List[str]
    search_results: List[Dict[str, str]]  # {title, url, content}
    report_goal: str
    report: str
    goal_achieved: bool
    iterations: int

def _dedup_results(items: List[Dict[str, str]]) -> List[Dict[str, str]]:
    seen = set()
    out = []
    for it in items:
        key = (it.get("url") or "").strip() or (it.get("title") or "").strip() or (it.get("content") or "")[:120]
        if key and key not in seen:
            seen.add(key)
            out.append(it)
    return out

def _limit_max(items: List[Dict[str, str]], max_n: int = 10) -> List[Dict[str, str]]:
    return items[:max_n] if len(items) > max_n else items

def _format_results_for_prompt(results: List[Dict[str, str]]) -> str:
    lines = []
    for i, r in enumerate(results, 1):
        lines.append(
            f"[{i}] {r.get('title','(제목 없음)')}\nURL: {r.get('url','')}\n{(r.get('content','') or '')[:1200]}\n"
        )
    return "\n".join(lines)

def _trueish(s: str) -> bool:
    s = (s or "").strip().lower()
    return s.startswith("t") or s in {"yes", "y", "1", "true", "참", "예", "네"}

### 노드 정의

In [6]:
# 검색 필요 여부 판단
def assess_search_need_node(state: GraphState) -> Dict[str, Any]:
    question = state["question"]

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "당신은 사용자의 질문에 웹 검색이 필요한지 여부를 판정하는 엄격한 라우터입니다. "
         "오직 'True' 또는 'False'만 답변하세요. "
         "최신 사실, 가격, 법률, 최근 사건, 고유명사 검증, 외부 인용이 필요한 경우 True를 반환하세요. "
         "시간에 무관한 일반 지식이나 의견이라면 False를 반환하세요."),
        ("human", "질문: {question}\n웹 검색이 필요한가요? (True/False)")
    ])
    chain = prompt | llm
    resp = chain.invoke({"question": question}).content.strip()
    need_search = _trueish(resp)
    print(f"[판정] 검색 필요 여부 = {need_search}  (LLM 응답: {resp})")
    return {"need_search": need_search, "iterations": state.get("iterations", 0)}

In [7]:
# 검색 쿼리 생성
def generate_queries_node(state: GraphState) -> Dict[str, Any]:
    question = state["question"]

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "사용자의 질문에 답하기 위한 웹 검색 쿼리를 구체적으로 생성하세요. "
         "중복 없이 1~3개 생성하고, 각 줄에 하나씩 작성하세요."),
        ("human", "현재 날짜 기준(Asia/Seoul).\n질문: {question}\n검색 쿼리:")
    ])
    chain = prompt | llm
    raw = chain.invoke({"question": question}).content
    queries = [q.strip("- •\t ").strip() for q in raw.split("\n") if q.strip()]
    queries = [q for q in queries if len(q) > 1][:3]
    if len(queries) < 2:
        queries = list(dict.fromkeys(queries + [question]))[:2]
    print(f"[쿼리 생성] {queries}")
    return {"queries": queries}

In [8]:
# 검색 실행
def search_node(state: GraphState) -> Dict[str, Any]:
    print("---검색 실행---")
    queries = state.get("queries") or [state["question"]]
    collected: List[Dict[str, str]] = []

    def _do_search(q: str, attempt: int = 0):
        try:
            res = tavily_tool.invoke({"query": q})
            for r in res.get("results", []):
                collected.append({
                    "title": r.get("title", "") or "",
                    "url": r.get("url", "") or "",
                    "content": r.get("content", "") or r.get("raw_content", "") or ""
                })
        except Exception as e:
            print(f"[검색 오류] {e}")
            if attempt == 0:
                time.sleep(1.2)
                _do_search(q, attempt=1)

    for q in queries:
        if len(collected) >= 10:
            break
        _do_search(q)

    collected = _dedup_results(collected)

    if len(collected) < 2:
        _do_search(state["question"])
        collected = _dedup_results(collected)

    collected = _limit_max(collected, max_n=10)
    print(f"[검색 결과] {len(collected)}개 수집 완료")
    return {"search_results": collected}

In [9]:
# 보고서 목표 생성
def generate_report_goal_node(state: GraphState) -> Dict[str, Any]:
    print("---보고서 목표 생성---")
    question = state["question"]
    search_results = state.get("search_results", [])

    prompt = ChatPromptTemplate.from_messages([
        ("system", "간결하고 검증 가능한 보고서 목표를 정의하세요. "
                   "범위, 핵심 질문, 전달물 형식, 성공 기준을 1~3문장으로 작성하세요."),
        ("human", "질문: {question}\n\n자료:\n{sources}\n\n목표만 작성하세요.")
    ])
    chain = prompt | llm
    goal = chain.invoke({
        "question": question,
        "sources": _format_results_for_prompt(search_results) if search_results else "(외부 자료 없음)"
    }).content.strip()

    print(f"[목표] {goal}")
    return {"report_goal": goal}

In [10]:

# 보고서 작성
def synthesize_report_node(state: GraphState) -> Dict[str, Any]:
    print("---보고서 작성---")
    question = state["question"]
    results = state.get("search_results", [])

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "간결하고 증거 기반의 보고서를 작성하세요. "
         "구성: 개요, 발견사항, 분석, 한계, 결론. "
         "출처는 [n] 형식으로 인용하고, 불확실성은 명확히 언급하세요."),
        ("human", "질문: {question}\n\n자료:\n{sources}\n\n보고서를 작성하세요.")
    ])
    chain = prompt | llm
    report = chain.invoke({
        "question": question,
        "sources": _format_results_for_prompt(results) if results else "(외부 자료 없음)"
    }).content.strip()

    return {"report": report}

In [11]:
# 목표 달성 여부 판단
def check_goal_achieved_node(state: GraphState) -> Dict[str, Any]:
    print("---목표 달성 여부 판단---")
    report_goal = state.get("report_goal", "")
    report = state.get("report", "")
    sources = _format_results_for_prompt(state.get("search_results", []))

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "보고서가 목표를 달성했는지 판단하세요. "
         "엄격하게 'True' 또는 'False'만 반환하세요."),
        ("human",
         "보고서 목표:\n{goal}\n\n보고서:\n{report}\n\n출처:\n{sources}\n\n달성 여부:")
    ])
    chain = prompt | llm
    resp = chain.invoke({"goal": report_goal, "report": report, "sources": sources}).content.strip()
    achieved = _trueish(resp)
    print(f"[달성 여부] {achieved} (원본 응답: {resp})")
    return {"goal_achieved": achieved}

In [12]:
# 쿼리 정련
def refine_queries_node(state: GraphState) -> Dict[str, Any]:
    print("---쿼리 정련---")
    report_goal = state.get("report_goal", "")
    report = state.get("report", "")

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "목표와 현재 보고서를 바탕으로, 증거 부족을 해소할 수 있는 후속 검색 쿼리 최대 3개를 제안하세요."),
        ("human", "목표:\n{goal}\n\n현재 보고서:\n{report}\n\n쿼리:")
    ])
    chain = prompt | llm
    raw = chain.invoke({"goal": report_goal, "report": report}).content
    new_queries = [q.strip("- •\t ").strip() for q in raw.split("\n") if q.strip()]
    new_queries = [q for q in new_queries if len(q) > 1][:3]

    prev = state.get("queries", [])
    merged = list(dict.fromkeys(prev + new_queries))
    print(f"[정련된 쿼리] {new_queries}")
    return {"queries": merged, "iterations": state.get("iterations", 0) + 1}



In [13]:
def route_need_search(state: GraphState) -> str:
    return "search" if state.get("need_search") else "skip"

def route_goal(state: GraphState) -> str:
    return "done" if state.get("goal_achieved") else "not_done"

def route_continue_or_stop(state: GraphState) -> str:
    if state.get("iterations", 0) >= 3:
        return "stop"
    if len(state.get("search_results", [])) >= 10:
        return "stop"
    return "continue"

### Graph

In [14]:
workflow = StateGraph(GraphState)

workflow.add_node("assess_search_need", assess_search_need_node)
workflow.add_node("generate_queries", generate_queries_node)
workflow.add_node("search", search_node)
workflow.add_node("generate_report_goal", generate_report_goal_node)
workflow.add_node("synthesize_report", synthesize_report_node)
workflow.add_node("check_goal_achieved", check_goal_achieved_node)
workflow.add_node("refine_queries", refine_queries_node)

workflow.set_entry_point("assess_search_need")

workflow.add_conditional_edges(
    "assess_search_need",
    route_need_search,
    {"search": "generate_queries", "skip": "generate_report_goal"},
)

workflow.add_edge("generate_queries", "search")
workflow.add_edge("search", "generate_report_goal")
workflow.add_edge("generate_report_goal", "synthesize_report")
workflow.add_edge("synthesize_report", "check_goal_achieved")

workflow.add_conditional_edges(
    "check_goal_achieved",
    route_goal,
    {"done": END, "not_done": "refine_queries"},
)

workflow.add_conditional_edges(
    "refine_queries",
    route_continue_or_stop,
    {"continue": "search", "stop": "synthesize_report"},
)

app = workflow.compile()

### Test Code

In [15]:
question = "이번 시즌 삼성라이온즈 경기력 평가 보고서 작성해줘"
result_state = app.invoke({"question": question})

print("\n\n===== 보고서 =====\n")
print(result_state.get("report", ""))
print("\n===== 목표 =====\n")
print(result_state.get("report_goal", ""))
print("\n===== 달성 여부 =====\n")
print(result_state.get("goal_achieved", ""))

[판정] 검색 필요 여부 = True  (LLM 응답: True)
[쿼리 생성] ['2024 삼성라이온즈 시즌 경기력 평가', '2024 KBO 삼성라이온즈 팀 분석', '2024 삼성라이온즈 성적 및 선수별 활약']
---검색 실행---
[검색 결과] 6개 수집 완료
---보고서 목표 생성---
[목표] 보고서 목표:  
2024시즌 삼성 라이온즈의 경기력을 정량적·정성적으로 평가하여, 정규시즌 및 포스트시즌 성과, 주요 전력 변화, 강·약점, 성공 요인을 분석한다.  
범위는 2024년 정규시즌 및 포스트시즌 전체이며, 핵심 질문은 "삼성 라이온즈가 예상을 깨고 준우승에 오른 주요 원동력과 한계는 무엇인가?"이다.  
전달물은 표와 그래프를 포함한 요약 보고서(PDF)이며, 성공 기준은 주요 성과와 개선점을 명확히 도출하고, 객관적 데이터와 비교 분석을 통해 구체적 인사이트를 제공하는 것이다.
---보고서 작성---
---목표 달성 여부 판단---
[달성 여부] True (원본 응답: True)


===== 보고서 =====

### 2024 시즌 삼성 라이온즈 경기력 평가 보고서

#### 1. 개요
2024년 삼성 라이온즈는 전문가들의 예상을 뒤엎고 정규시즌 2위(78승 2무 64패, 승률 0.549)를 기록하며 3년 만에 포스트시즌에 진출, 9년 만에 한국시리즈에 올랐다. 최종적으로 준우승에 머물렀으나, 전년도 8위에서의 반등과 팀 컬러의 변화, 세대교체 등 여러 측면에서 의미 있는 시즌을 보냈다[1][4][6].

#### 2. 발견사항
- **정규시즌 성적**: 2위(78승 2무 64패, 승률 0.549)[1][3][4]
- **홈/원정 승률**: 홈 0.562(41승 32패), 원정 0.536(37승 2무 32패)로 큰 차이 없이 안정적[1]
- **타격**: 팀 타율 0.269(9위), 홈런 185개(1위), 장타율 0.428(3위), OPS 0.774(5위)[1][3][6]
- **투수**: 팀 평균자책점 4.68(3위), WHIP 1.45